<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-GPT_Python-2023/blob/main/HTML-02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install playwright > /dev/null
!pip install  lxml

In [4]:
!playwright install

146.4 Mb [] 0% 9.8s146.4 Mb [] 0% 126.5s146.4 Mb [] 0% 99.3s146.4 Mb [] 0% 67.2s146.4 Mb [] 0% 41.0s146.4 Mb [] 0% 24.7s146.4 Mb [] 1% 11.3s146.4 Mb [] 1% 10.0s146.4 Mb [] 2% 8.7s146.4 Mb [] 2% 7.7s146.4 Mb [] 3% 7.0s146.4 Mb [] 3% 6.4s146.4 Mb [] 4% 6.2s146.4 Mb [] 4% 5.8s146.4 Mb [] 6% 4.9s146.4 Mb [] 6% 4.6s146.4 Mb [] 7% 4.2s146.4 Mb [] 8% 4.0s146.4 Mb [] 9% 3.8s146.4 Mb [] 11% 3.4s146.4 Mb [] 12% 3.3s146.4 Mb [] 12% 3.1s146.4 Mb [] 13% 3.1s146.4 Mb [] 14% 3.1s146.4 Mb [] 15% 2.8s146.4 Mb [] 17% 2.7s146.4 Mb [] 18% 2.6s146.4 Mb [] 20% 2.4s146.4 Mb [] 21% 2.3s146.4 Mb [] 22% 2.2s146.4 Mb [] 24% 2.1s146.4 Mb [] 25% 2.0s146.4 Mb [] 26% 1.9s146.4 Mb [] 27% 1.9s146.4 Mb [] 28% 1.8s146.4 Mb [] 30% 1.8s146.4 Mb [] 31% 1.7s146.4 Mb [] 32% 1.6s146.4 Mb [] 33% 1.6s146.4 Mb [] 34% 1.6s146.4 Mb [] 35% 1.5s146.4 Mb [] 36% 1.5s146.4 Mb [] 38% 1.5s146.4 Mb [] 39% 1.4s146.4 Mb [] 40% 1.4s146.4 Mb [] 42% 1.3s146.4 Mb [] 43% 1.3s146.4 Mb [] 45% 1.2s146.4 Mb [] 46% 1.2s146.4 Mb [] 48% 1.1s146.4 Mb []

In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.1 MB/s eta 0:00:00


In [6]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.
)

In [7]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

In [ ]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

In [9]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

In [11]:
await navigate_tool.arun(
    {"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"}
)

'Navigating to https://web.archive.org/web/20230428131116/https://www.cnn.com/world returned status code 200'

In [12]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun(
    {"selector": ".container__headline", "attributes": ["innerText"]}
)

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [13]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name["current_webpage"].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

### Use within an Agent

In [ ]:
!pip install openai

In [24]:
import os
os.environ["OPENAI_API_KEY"] = "sk"

In [25]:
from langchain.llms import OpenAI

In [26]:
from langchain.agents import initialize_agent, AgentType
# from langchain.chat_models import ChatAnthropic

# llm = ChatAnthropic(temperature=0)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()
llm = OpenAI(temperature=0)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
result = await agent_chain.arun("What are the headers on langchain.com?")
print(result)